# Importing libraries

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Setting up Hyperparameters

1.   **batch_size:** how many independent sequences will we process in parallel?
2.   **List item:** what is the maximum context length for predictions?
3. **max_iters:** The total number of training steps the model will go through.
4. **eval_interval:** The model will evaluate its performance (loss) on the training and validation sets every 100 iterations.
5. **learning_rate:** The step size used during optimization. It controls how quickly the model adjusts its parameters.







In [ ]:
batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0


torch.manual_seed(1337)

In [ ]:
file_path = '/content/Transformer Dataset.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

# Loading Dataset

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# loss function
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Self Attention and Transformer Block



1.   **class Head(nn.Module):** Defines a single "head" of the self-attention mechanism. This head is responsible for focusing on a specific aspect of the input sequence.
2. **class MultiHeadAttention(nn.Module):** Combines multiple attention heads in parallel, allowing the model to attend to different features of the input.
3. **class FeedFoward(nn.Module):** A simple feedforward network (a series of linear layers and activation functions) that takes the output of the self-attention layers and processes it further.
4. **class Block(nn.Module):** Encapsulates the entire Transformer block. A block consists of a self-attention layer (communication) followed by a feedforward layer (computation).



In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# Bigram Model

A bigram language model is a statistical method for predicting the next word (or character) in a sequence based on the immediately preceding word (or character).

In [ ]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

# Pytorch Optimizer

The PyTorch optimizer is a tool that adjusts the parameters of the bigram language model during training to minimize its prediction errors.

In [ ]:
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.213212 M parameters
step 0: train loss 4.6987, val loss 4.7023
step 100: train loss 2.6463, val loss 2.6621
step 200: train loss 2.5004, val loss 2.5299
step 300: train loss 2.4216, val loss 2.4591
step 400: train loss 2.3462, val loss 2.3989
step 500: train loss 2.2901, val loss 2.3297
step 600: train loss 2.2334, val loss 2.2703
step 700: train loss 2.2056, val loss 2.2414
step 800: train loss 2.1370, val loss 2.1817
step 900: train loss 2.0942, val loss 2.1556
step 1000: train loss 2.0849, val loss 2.1347
step 1100: train loss 2.0394, val loss 2.1111
step 1200: train loss 2.0142, val loss 2.0969
step 1300: train loss 2.0054, val loss 2.0743
step 1400: train loss 1.9869, val loss 2.0592
step 1500: train loss 1.9556, val loss 2.0370
step 1600: train loss 1.9451, val loss 2.0268
step 1700: train loss 1.9181, val loss 2.0144
step 1800: train loss 1.9133, val loss 1.9918
step 1900: train loss 1.9107, val loss 1.9763
step 2000: train loss 1.8860, val loss 1.9862
step 2100: train loss 1.

# Text Generation

In [ ]:
# generate from the model

prompt = "Hello World"
context = torch.tensor(encode(prompt), dtype=torch.long, device=device).unsqueeze(0)  # (1, T) shape
generated_text_idx = model.generate(context, max_new_tokens=2000)  # Generate 200 new tokens
generated_text = decode(generated_text_idx[0].tolist())

print("You: ", prompt)
print("GPT: ", generated_text)

You:  Hello World
GPT:  Hello World’s belthing pryspeasally, have pinsplice got outside him, stame. ”
“Miny befoalding you had glarid to head are had slood-severy’d lide the, and his not bare twelented to in him keps, was back out their wonderert that I tholl warts got in if blow-Kiss to detteringer
“every Stress’s lyark sleetly.” Barning for out . . . . . atter that selept just. For, must on they ground him sksing. He much are explase-— as I	way and the settly udddood, like thhing your caup to Castany: Put as Stannis Ned.
It — — Seriestly Pribe’s botty. The strassenter and to.”
“Wark, as Lanthing to go chardled — Pythrouge in a face fave on a throe membents and the revaging, she agame, why might her and out to it weart,” speapsup Hermione priess the scoad. “I she he’d cleasling fire and dammanded corrice. They . . . he’ve sade feath a tap wergelf.”
“Uhe through with hardn’t as Hogwart Ron’s gon.

Wrond her quited clild, incoldiang to his talturing than soner his abild his esight of at